This notebook is developed as a documentation for experimentation and development of a model for the sacred.ai project created by Vasu Jain, Amee Madhani, Anand Chauhan, and Ananya Chauhan as part of our university course. The project to create one or multiple model in order to answer question based on life, the universe, and everything with the knowledge of several religious texts for Hinduism, Christianity, and Islam.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.config.list_physical_devices())

2023-03-23 00:45:47.350867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-03-23 00:45:49.530671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-23 00:45:49.601297: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-23 00:45:49.601377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import spacy
# verifying if cuda setup works properly
spacy.prefer_gpu()

True

Current working approaches include looking to religious semantic searching and question-answering. The latter of these requires an extensive dataset covering a wide variety of questions and associated answers and outputs we'd be looking for, and is therefore hard to implement since we do not know if anything like that even exists (hopefully not lol)

So we'd have to play around a lot with feature extraction, semantics and transformers and their encoding.e

Can't do regression based linear transformations either since that, too, requires a dataset.

Let's just import all our datasets for now.

In [3]:
bhagvadgita = pd.read_csv('../dataset/gitaDataset.csv')
quran = pd.read_csv('../dataset/quranDataset.csv')
bible = pd.read_csv('../dataset/bibleDataset.csv')
print(bhagvadgita.shape)
bhagvadgita.head()

(700, 4)


,Title,Chapter,Verse,English Translation
0,Arjuna's Vishada Yoga,Chapter 1,Verse 1.1,"Dhrtarashtra asked of Sanjaya: O SANJAYA, what..."
1,Arjuna's Vishada Yoga,Chapter 1,Verse 1.2,Sanjaya explained: Now seeing that the army of...
2,Arjuna's Vishada Yoga,Chapter 1,Verse 1.3,"Behold O, Master, the mighty army of the sons ..."
3,Arjuna's Vishada Yoga,Chapter 1,Verse 1.4,"Present here are the mighty archers, peers or ..."
4,Arjuna's Vishada Yoga,Chapter 1,Verse 1.5,"Dhrishtaketu, Chekitana, and the valiant king ..."


In [4]:
bhagvadgita['Title'].unique()

array(["Arjuna's Vishada Yoga", 'Sankhya Yoga', 'Karma Yoga',
       'Jnana-Karma-Sanyasa Yoga', 'Atma-Samyama Yoga',
       'Jnana-Vijnana Yoga', 'Aksara-ParaBrahma Yoga',
       'Raja-Vidya-Raja-Guhya Yoga', 'Vibhuti Yoga',
       'Viswarupa-Darsana Yoga', 'Bhakti Yoga',
       'Ksetra-Ksetrajna-Vibhaga Yoga', 'Gunatraya-Vibhaga Yoga',
       'Purushottama Yoga', 'Daivasura-Sampad-Vibhaga Yoga',
       'Shraddhatraya-Vibhaga Yoga', 'Moksha-Sanyasa Yoga'], dtype=object)

In [5]:
print(quran.shape)
quran.head()

(6236, 5)


,Name,Surah,Ayat,Verse,Tafseer
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful
1,The Opening,1,2,"Praise be to Allah, Lord of the Worlds,",In the Name of God the name of a thing is that...
2,The Opening,1,3,"The Beneficent, the Merciful.",The Compassionate the Merciful that is to say ...
3,The Opening,1,4,"Owner of the Day of Judgment,",Master of the Day of Judgement that is the day...
4,The Opening,1,5,Thee (alone) we worship; Thee (alone) we ask f...,You alone we worship and You alone we ask for ...


In [6]:
print(bible.shape)
bible.head()

(31103, 5)


,field,book,chapter,verse,text
0,1001001,1,1,1,At the first God made the heaven and the earth.
1,1001002,1,1,2,And the earth was waste and without form; and ...
2,1001003,1,1,3,"And God said, Let there be light: and there wa..."
3,1001004,1,1,4,"And God, looking on the light, saw that it was..."
4,1001005,1,1,5,"Naming the light, Day, and the dark, Night. An..."


In [7]:
# whatever approach we implement now we will implement first on the bhagvadgita, since it is the smallest.
# everything else will be taken care of later lmao
try:
    # removing useless text and making everthing integer.
    bhagvadgita['Verse'] = bhagvadgita['Verse'].apply(lambda x: x.split('.')[-1]).astype('int')
    bhagvadgita['Chapter'] = bhagvadgita['Chapter'].apply(lambda x: x.split()[-1]).astype('int')
except AttributeError:
    # circumventing inplace problems
    pass
# check dtypes
bhagvadgita.dtypes

Title                  object
Chapter                 int64
Verse                   int64
English Translation    object
dtype: object

In [8]:
# finding the maximum amonut of words in any single shlok/verse/passage. we will model our max query length around it. maybe.
# every book will therfore contain three tensors. max pooled, mean pooled, and all vectors with a lot of zeroes.
max_length = 0
for i in bhagvadgita['English Translation']:
    if len(i.split()) > max_length:
        max_length = len(i.split())
print(max_length)
for j in bible['text']:
    if len(j.split()) > max_length:
        max_length = len(j.split())
print(max_length)
for k in quran['Verse']:
    if len(k.split()) > max_length:
        max_length = len(k.split())
print(max_length)
for l in quran['Tafseer'].astype('str'):
    if len(l.split()) > max_length:
        max_length = len(l.split())
print(max_length)

118
118
273
584


In [9]:
# MAX_LENGTH = 584 
# best case, let's try smaller sizes first
MAX_LENGTH = 256

In [10]:
# loading the word2vec model that we'll be using for now.
nlp = spacy.load('en_core_web_lg')

In [11]:
# investigating if the standard vector function does max or mean pooling.
lmaobase = np.array([token.vector.get() for token in nlp(bhagvadgita['English Translation'][0])])
lmao = lmaobase.mean(axis=(0))
argm = np.array([float(token.vector_norm) for token in nlp(bhagvadgita['English Translation'][0])])
lmao1 = lmaobase[argm.argmax()]
lmao2 = np.array(nlp(bhagvadgita['English Translation'][0]).vector.get())
lmao3 = lmaobase.max(axis=(0))
print(f"Max v/s overall vector: {np.array_equal(lmao1, lmao2)} \nMean v/s overall vector: {np.array_equal(lmao, lmao2)} \nMax v/s Argmax vector: {np.array_equal(lmao1, lmao3)}")

Max v/s overall vector: False 
Mean v/s overall vector: True 
Max v/s Argmax vector: False


In [12]:
# experimenting with how to make a vector with all zeroes
# find a zero vector
zerosample = np.array(nlp(bhagvadgita['English Translation'][0].split()[0]).vector.get())
constructedzeros = np.zeros((300,))
np.array_equal(constructedzeros, zerosample)
zerosample = np.array([token.vector.get() for token in nlp(bhagvadgita['English Translation'][0])])
constructarr = np.zeros((MAX_LENGTH - zerosample.shape[0], 300))
hope = np.append(zerosample, constructarr, axis=(0))
hope.shape

(256, 300)

# IT WORKS!!

With this we can see that spacy's `en_core_web_lg` uses mean pooling. so we can make our data as follows:

In [13]:
# test with one field, trying both max and mean pooling.
maxtokens = []
meantokens = []
alltokens = []

for i in bhagvadgita['English Translation']:
    doc = nlp(i)
    tokenlist = np.array([token.vector.get() for token in doc])
    maxnormlist = np.array([float(token.vector_norm) for token in doc])
    if MAX_LENGTH - tokenlist.shape[0] > 0:
        constructarr = np.zeros((MAX_LENGTH - tokenlist.shape[0], 300))
        alltokens.append(np.append(tokenlist, constructarr, axis=(0)))
    else:
        alltokens.append(tokenlist[:MAX_LENGTH])

    maxtokens.append(np.array(tokenlist[maxnormlist.argmax()]))
    meantokens.append(np.array(tokenlist.mean(axis=(0))))

tmaxtokens = tf.convert_to_tensor(maxtokens)
tmeantokens = tf.convert_to_tensor(meantokens)
talltokens = tf.convert_to_tensor(alltokens)

tmaxtokens.shape, tmeantokens.shape, talltokens.shape

2023-03-23 00:46:40.547638: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 00:46:40.552152: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-23 00:46:40.552257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-23 00:46:40.552286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

(TensorShape([700, 300]),
 TensorShape([700, 300]),
 TensorShape([700, 256, 300]))

In [14]:
# converting this to a function for later use:
def tensorgoBrr(df: pd.DataFrame, column: str):
    """this function returns tensors for max pooled, mean pooled, and all present (padded up or down to 256 vectors)
    the return is a tuple with the order max, mean, and all tokens respectively."""
    maxtokens = []
    meantokens = []
    alltokens = []

    for i in df[column]:
        doc = nlp(i)
        tokenlist = np.array([token.vector.get() for token in doc])
        maxnormlist = np.array([float(token.vector_norm) for token in doc])
        if MAX_LENGTH - tokenlist.shape[0] > 0:
            constructarr = np.zeros((MAX_LENGTH - tokenlist.shape[0], 300))
            alltokens.append(np.append(tokenlist, constructarr, axis=(0)))
        else:
            alltokens.append(tokenlist[:MAX_LENGTH])

        maxtokens.append(np.array(tokenlist[maxnormlist.argmax()]))
        meantokens.append(np.array(tokenlist.mean(axis=(0))))

    tmaxtokens = tf.convert_to_tensor(maxtokens)
    tmeantokens = tf.convert_to_tensor(meantokens)
    talltokens = tf.convert_to_tensor(alltokens)

    return (tmaxtokens, tmeantokens, talltokens)

We have now created the tensors for all verses of the bhagvadgita. for the demo this is all that we wil be doing.

Bear in mind that the tensor with all adjusted vectors for the Bhagvadgita alone is over 400 MB, which, considering my system VRAM of 8GB, might become a computational power problem later down the line.

In [15]:
# importing all AI experimentation libraries.
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import keras
from keras import layers
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed, Reshape, Flatten
from keras.utils import plot_model

In [16]:
timesteps = talltokens.shape[1]  # Length of your sequences
input_dim = talltokens.shape[2]
latent_dim = 64

input_img = keras.Input(shape=(256, 300))

x = layers.Conv1D(300/5, (5), activation='tanh', padding='same')(input_img)
# x = layers.MaxPooling1D((8), padding='same')(x)
x = layers.Conv1D(60/3, (3), activation='sigmoid', padding='same')(x)
# x = layers.MaxPooling1D((8), padding='same')(x)
x = layers.Conv1D(20/2, (3), activation='relu', padding='same')(x)
# encoded = layers.MaxPooling1D((4), padding='same')(x)
encoded = x

x = layers.Conv1D(20, (3), activation='tanh', padding='same')(encoded)
x = layers.UpSampling1D((1))(x)
x = layers.Conv1D(60, (3), activation='sigmoid', padding='same')(x)
x = layers.UpSampling1D((1))(x)
x = layers.Conv1D(300, (1), activation='sigmoid')(x)
x = layers.UpSampling1D((1))(x)
decoded = layers.Conv1D(300, (5), activation='relu', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss=['cosine_similarity'], metrics=['mse', 'cosine_similarity'])
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 300)]        0         
                                                                 
 conv1d (Conv1D)             (None, 256, 60)           90060     
                                                                 
 conv1d_1 (Conv1D)           (None, 256, 20)           3620      
                                                                 
 conv1d_2 (Conv1D)           (None, 256, 10)           610       
                                                                 
 conv1d_3 (Conv1D)           (None, 256, 20)           620       
                                                                 
 up_sampling1d (UpSampling1D  (None, 256, 20)          0         
 )                                                               
                                                             

In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [18]:
history = autoencoder.fit(talltokens, talltokens, batch_size=2, epochs=75, shuffle=True, verbose=1, callbacks=tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=3))

Epoch 1/75


2023-03-23 00:46:50.210460: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 430080000 exceeds 10% of free system memory.
2023-03-23 00:46:52.540006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-03-23 00:46:54.287036: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-23 00:46:54.744058: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fee9291f960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-23 00:46:54.744107: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 Super, Compute Capability 7.5
2023-03-23 00:46:54.752169: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-23 00:46:54.980905: I tensorflow/tsl/platform/default/

350/350 [==============================] - 12s 13ms/step - loss: -0.0513 - mse: 6.0704 - cosine_similarity: 0.0513
Epoch 2/75
350/350 [==============================] - 4s 12ms/step - loss: -0.0607 - mse: 3.5921 - cosine_similarity: 0.0607
Epoch 3/75
350/350 [==============================] - 4s 11ms/step - loss: -0.0645 - mse: 3.3468 - cosine_similarity: 0.0645
Epoch 4/75
350/350 [==============================] - 4s 11ms/step - loss: -0.0674 - mse: 3.3237 - cosine_similarity: 0.0674
Epoch 5/75
350/350 [==============================] - 4s 11ms/step - loss: -0.0702 - mse: 3.3187 - cosine_similarity: 0.0702
Epoch 6/75
350/350 [==============================] - 4s 11ms/step - loss: -0.0729 - mse: 3.3720 - cosine_similarity: 0.0729
Epoch 7/75
350/350 [==============================] - 3s 10ms/step - loss: -0.0746 - mse: 3.3682 - cosine_similarity: 0.0746
Epoch 8/75
350/350 [==============================] - 4s 11ms/step - loss: -0.0758 - mse: 3.3709 - cosine_similarity: 0.0758
Epoch 9/75

That model performance is unsatisfactory, with a cosine similarity of `~0.103`, it goes rather far from our input vector.
WE will now try deep neural autoencoding instead, on all three of our tensors (max, mean, all)

## BUT.
our current hypothesis is that if we scale all our word embeddings like this for just compression's sake then maybe similarity won't matter when we try to match them.
(in case this approach also does not work)

In [19]:
# trying again but with a deep neural network this time.
input_imgmax = keras.Input(shape=(300))
dencoded = Flatten()(input_imgmax)
dencoded = Dense(units=300/2, activation='relu')(dencoded)
dencoded = Dense(units=300/4, activation='relu')(dencoded)
dencoded = Dense(units=300/12, activation='relu')(dencoded)
dencoded = Dense(units=300/60, activation='relu')(dencoded)

ddecoded = Dense(units=300/12, activation='relu')(dencoded)
ddecoded = Dense(units=300/4, activation='relu')(ddecoded)
ddecoded = Dense(units=300/2, activation='sigmoid')(ddecoded)
ddecoded = Dense(units=300, activation='sigmoid')(ddecoded)

dmaxautoencoder = keras.Model(input_imgmax, ddecoded)
dmaxautoencoder.compile(optimizer='adam', loss=['cosine_similarity'], metrics=['mse', 'cosine_similarity'])
dmaxautoencoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300)]             0         
                                                                 
 flatten (Flatten)           (None, 300)               0         
                                                                 
 dense (Dense)               (None, 150)               45150     
                                                                 
 dense_1 (Dense)             (None, 75)                11325     
                                                                 
 dense_2 (Dense)             (None, 25)                1900      
                                                                 
 dense_3 (Dense)             (None, 5)                 130       
                                                                 
 dense_4 (Dense)             (None, 25)                150 

In [20]:
history = dmaxautoencoder.fit(tmaxtokens, tmaxtokens, batch_size=2, epochs=50, shuffle=True, verbose=1, callbacks=tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=3))

Epoch 1/50
350/350 [==============================] - 4s 6ms/step - loss: -0.3247 - mse: 57.1899 - cosine_similarity: 0.3247
Epoch 2/50
350/350 [==============================] - 2s 5ms/step - loss: -0.4964 - mse: 57.5034 - cosine_similarity: 0.4964
Epoch 3/50
350/350 [==============================] - 2s 5ms/step - loss: -0.5491 - mse: 57.7563 - cosine_similarity: 0.5491
Epoch 4/50
350/350 [==============================] - 2s 5ms/step - loss: -0.5666 - mse: 57.7555 - cosine_similarity: 0.5666
Epoch 5/50
350/350 [==============================] - 2s 5ms/step - loss: -0.5910 - mse: 57.6627 - cosine_similarity: 0.5910
Epoch 6/50
350/350 [==============================] - 2s 5ms/step - loss: -0.6013 - mse: 57.7333 - cosine_similarity: 0.6013
Epoch 7/50
350/350 [==============================] - 2s 5ms/step - loss: -0.6090 - mse: 57.7385 - cosine_similarity: 0.6090
Epoch 8/50
350/350 [==============================] - 2s 5ms/step - loss: -0.5963 - mse: 57.7616 - cosine_similarity: 0.5963


this is miles better, but the max pooling does mean there is a not insignificant loss of data here. will try flattening top 8 vectors and training the model

In [21]:
xdproc = layers.MaxPooling1D((32), padding='same')(input_img)
xdp = keras.Model(input_img, xdproc)
inpprep = xdp.predict(talltokens)

22/22 [==============================] - 0s 10ms/step


In [22]:

xdinput = keras.Input(shape=(8, 300))

xdencoded = Flatten()(xdinput)
xdencoded = Dense(units=300*4, activation='relu')(xdencoded)
xdencoded = Dense(units=300/2, activation='relu')(xdencoded)
xdencoded = Dense(units=300/4, activation='relu')(xdencoded)
xdencoded = Dense(units=300/12, activation='relu')(xdencoded)
xdencoded = Dense(units=300/60, activation='relu')(xdencoded)

xddecoded = Dense(units=300/12, activation='relu')(xdencoded)
xddecoded = Dense(units=300/4, activation='relu')(xddecoded)
xddecoded = Dense(units=300/2, activation='sigmoid')(xddecoded)
xddecoded = Dense(units=300, activation='sigmoid')(xddecoded)
xddecoded = Dense(units=300*8, activation='relu')(xddecoded)
xddecoded = Reshape((8, 300))(xddecoded)

xdmaxautoencoder = keras.Model(xdinput, xddecoded)
xdmaxautoencoder.compile(optimizer='adam', loss=['cosine_similarity'], metrics=['mse', 'cosine_similarity'])
xdmaxautoencoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8, 300)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 2400)              0         
                                                                 
 dense_8 (Dense)             (None, 1200)              2881200   
                                                                 
 dense_9 (Dense)             (None, 150)               180150    
                                                                 
 dense_10 (Dense)            (None, 75)                11325     
                                                                 
 dense_11 (Dense)            (None, 25)                1900      
                                                                 
 dense_12 (Dense)            (None, 5)                 130 

In [23]:
history = xdmaxautoencoder.fit(inpprep, inpprep, batch_size=2, epochs=50, shuffle=True, verbose=1, callbacks=tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=3))

Epoch 1/50
349/350 [============================>.] - ETA: 0s - loss: -0.1697 - mse: 15.6377 - cosine_similarity: 0.1697

2023-03-23 00:53:21.796972: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 691200000 exceeds 10% of free system memory.


350/350 [==============================] - 6s 8ms/step - loss: -0.1700 - mse: 15.6471 - cosine_similarity: 0.1700
Epoch 2/50
350/350 [==============================] - 2s 7ms/step - loss: -0.1723 - mse: 14.6201 - cosine_similarity: 0.1723
Epoch 3/50
350/350 [==============================] - 2s 6ms/step - loss: -0.1737 - mse: 13.9579 - cosine_similarity: 0.1737
Epoch 4/50
347/350 [============================>.] - ETA: 0s - loss: -0.1741 - mse: 13.5398 - cosine_similarity: 0.1741

2023-03-23 00:53:29.394679: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 691200000 exceeds 10% of free system memory.


350/350 [==============================] - 3s 8ms/step - loss: -0.1741 - mse: 13.5475 - cosine_similarity: 0.1741
Epoch 5/50
350/350 [==============================] - 2s 7ms/step - loss: -0.1747 - mse: 13.2638 - cosine_similarity: 0.1747
Epoch 6/50
350/350 [==============================] - 2s 7ms/step - loss: -0.1748 - mse: 13.0566 - cosine_similarity: 0.1748
Epoch 7/50
346/350 [============================>.] - ETA: 0s - loss: -0.1753 - mse: 13.0048 - cosine_similarity: 0.1753

2023-03-23 00:53:37.163099: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 691200000 exceeds 10% of free system memory.


350/350 [==============================] - 3s 9ms/step - loss: -0.1752 - mse: 13.0040 - cosine_similarity: 0.1752
Epoch 8/50
350/350 [==============================] - 3s 8ms/step - loss: -0.1756 - mse: 12.9342 - cosine_similarity: 0.1756
Epoch 9/50
350/350 [==============================] - 3s 7ms/step - loss: -0.1759 - mse: 12.9939 - cosine_similarity: 0.1759
Epoch 10/50
350/350 [==============================] - 3s 9ms/step - loss: -0.1761 - mse: 13.0816 - cosine_similarity: 0.1761
Epoch 11/50
350/350 [==============================] - 2s 7ms/step - loss: -0.1762 - mse: 13.2833 - cosine_similarity: 0.1762
Epoch 12/50
350/350 [==============================] - 2s 6ms/step - loss: -0.1774 - mse: 13.4820 - cosine_similarity: 0.1774
Epoch 13/50
350/350 [==============================] - 3s 8ms/step - loss: -0.1786 - mse: 13.5378 - cosine_similarity: 0.1786
Epoch 14/50
350/350 [==============================] - 2s 7ms/step - loss: -0.1797 - mse: 13.7798 - cosine_similarity: 0.1797
Epoch 

so so far the max pooled one has worked the best. i wonder if i can get better results with mean, so i will rebuild the same model with mean.

In [24]:
dmeanautoencoder = keras.Model(input_imgmax, ddecoded)
dmeanautoencoder.compile(optimizer='adam', loss=['cosine_similarity'], metrics=['mse', 'cosine_similarity'])
dmeanautoencoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300)]             0         
                                                                 
 flatten (Flatten)           (None, 300)               0         
                                                                 
 dense (Dense)               (None, 150)               45150     
                                                                 
 dense_1 (Dense)             (None, 75)                11325     
                                                                 
 dense_2 (Dense)             (None, 25)                1900      
                                                                 
 dense_3 (Dense)             (None, 5)                 130       
                                                                 
 dense_4 (Dense)             (None, 25)                150 

In [25]:
history = dmeanautoencoder.fit(tmeantokens, tmeantokens, batch_size=2, epochs=50, shuffle=True, verbose=1, callbacks=tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=3))

Epoch 1/50
350/350 [==============================] - 4s 6ms/step - loss: -0.5846 - mse: 2.7308 - cosine_similarity: 0.5846
Epoch 2/50
350/350 [==============================] - 2s 6ms/step - loss: -0.6147 - mse: 2.7922 - cosine_similarity: 0.6147
Epoch 3/50
350/350 [==============================] - 2s 6ms/step - loss: -0.6189 - mse: 2.8326 - cosine_similarity: 0.6189
Epoch 4/50
350/350 [==============================] - 2s 6ms/step - loss: -0.6200 - mse: 2.8565 - cosine_similarity: 0.6200
Epoch 5/50
350/350 [==============================] - 2s 6ms/step - loss: -0.6227 - mse: 2.8809 - cosine_similarity: 0.6227
Epoch 6/50
350/350 [==============================] - 2s 6ms/step - loss: -0.6274 - mse: 2.8689 - cosine_similarity: 0.6274
Epoch 7/50
350/350 [==============================] - 2s 6ms/step - loss: -0.6275 - mse: 2.8717 - cosine_similarity: 0.6275
Epoch 8/50
350/350 [==============================] - 2s 6ms/step - loss: -0.6259 - mse: 2.8710 - cosine_similarity: 0.6259
Epoch 9/

Slightly better, but because of best practice concerns, we _MIGHT_ be using the max autoencoder for the demo.

In [26]:
# saving all the models, starting with the latest one
dmeanautoencoder.save('./models/bhagvadgita/mean_pooled_autoencoder.h5')

In [27]:
# saving the bad max pool model
xdmaxautoencoder.save('./models/bhagvadgita/top_8_max_pooled_autoencoder.h5')

In [28]:
# saving teh good max pool model
dmaxautoencoder.save('./models/bhagvadgita/BEST_max_pooled_autoencoder.h5')

In [29]:
# saving the conv1d model
autoencoder.save('./models/bhagvadgita/convolutional_autoencoder.h5')

now we can use the encoded values and save them with the dataframe

In [36]:
# splitting the model into just the encoder
ourencoder = keras.Model(dmeanautoencoder.input, dmeanautoencoder.layers[5].output)

In [39]:
bhagvadgita_encodings = ourencoder.predict(tmeantokens)

22/22 [==============================] - 0s 2ms/step


In [47]:
encodf = pd.DataFrame(bhagvadgita_encodings, columns=('a1','a2','a3','a4','a5'))
encodf.head()

,a1,a2,a3,a4,a5
0,1082.754639,896.849670,642.481567,0.000000,0.0
1,0.000000,723.449585,1098.451416,427.873535,0.0
2,0.000000,1376.264160,1331.519897,2368.946045,0.0
3,0.000000,1003.821289,1086.919678,1332.740845,0.0
4,0.000000,1400.724609,1457.762939,1932.723999,0.0


In [49]:
bhagvadgita_final = pd.concat([bhagvadgita, encodf], axis=1)
bhagvadgita_final.head()

,Title,Chapter,Verse,English Translation,a1,a2,a3,a4,a5
0,Arjuna's Vishada Yoga,1,1,"Dhrtarashtra asked of Sanjaya: O SANJAYA, what...",1082.754639,896.849670,642.481567,0.000000,0.0
1,Arjuna's Vishada Yoga,1,2,Sanjaya explained: Now seeing that the army of...,0.000000,723.449585,1098.451416,427.873535,0.0
2,Arjuna's Vishada Yoga,1,3,"Behold O, Master, the mighty army of the sons ...",0.000000,1376.264160,1331.519897,2368.946045,0.0
3,Arjuna's Vishada Yoga,1,4,"Present here are the mighty archers, peers or ...",0.000000,1003.821289,1086.919678,1332.740845,0.0
4,Arjuna's Vishada Yoga,1,5,"Dhrishtaketu, Chekitana, and the valiant king ...",0.000000,1400.724609,1457.762939,1932.723999,0.0


In [50]:
bhagvadgita_final.to_csv('../dataset/bhagvadgita_encoded.csv')